In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Simulation: We create a simulated game environment that generates player performance data based on player skill and difficulty.

In [3]:
def simulate_game(player_skill, difficulty):
    # Simplified simulation: higher skill, lower difficulty = better performance
    accuracy = np.random.normal(player_skill * (2 - difficulty), 0.2)
    completion_time = np.random.normal(10 / (player_skill + difficulty), 2)
    deaths = int(np.random.poisson(difficulty * (1 - player_skill/2)))
    return accuracy, completion_time, deaths



# Data Generation: We generate a dataset of simulated game sessions.

In [7]:
def generate_data(num_sessions=1000):
    data = []
    for _ in range(num_sessions):
        player_skill = np.random.uniform(0, 1)  # Skill level between 0 and 1
        difficulty = np.random.uniform(0, 1)  # Difficulty level between 0 and 1
        accuracy, completion_time, deaths = simulate_game(player_skill, difficulty)
        data.append([player_skill, difficulty, accuracy, completion_time, deaths])
    return pd.DataFrame(data, columns=['player_skill', 'difficulty', 'accuracy', 'completion_time', 'deaths'])

data = generate_data()

# Preprocessing: We create a performance score feature.

In [10]:
def preprocess_data(df):
    # Example: Create a performance score
    df['performance_score'] = df['accuracy'] * (1 / (df['completion_time'] + 1)) * (1 / (df['deaths'] + 1))
    return df

data = preprocess_data(data)

# Model Training: We train a Random Forest Regression model to predict the optimal difficulty level based on player skill and performance.

In [13]:
def train_model(df):
    X = df[['player_skill', 'accuracy', 'completion_time', 'deaths']]
    y = df['difficulty']  # Target: difficulty level

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse}")

    return model

model = train_model(data)

Mean Squared Error: 0.019687971446972312


# Difficulty Adjustment: The adjust_difficulty function uses the trained model to predict and adjust the difficulty level. We smooth the adjustment to avoid sudden changes.

In [16]:
def adjust_difficulty(player_skill, accuracy, completion_time, deaths, current_difficulty, model):
    input_data = pd.DataFrame([[player_skill, accuracy, completion_time, deaths]],
                              columns=['player_skill', 'accuracy', 'completion_time', 'deaths'])
    predicted_difficulty = model.predict(input_data)[0]

    # Smooth the adjustment to avoid abrupt changes
    adjusted_difficulty = (0.7 * predicted_difficulty + 0.3 * current_difficulty)

    # Ensure difficulty stays within valid range
    adjusted_difficulty = max(0, min(1, adjusted_difficulty))

    return adjusted_difficulty

# Gameplay Loop: We simulate a gameplay loop where the difficulty is dynamically adjusted after each round.

In [19]:
def gameplay_loop(model):
    player_skill = 0.5  # Initial player skill
    difficulty = 0.5  # Initial difficulty
    num_rounds = 50

    for round_num in range(num_rounds):
        accuracy, completion_time, deaths = simulate_game(player_skill, difficulty)
        print(f"Round {round_num + 1}:")
        print(f"  Accuracy: {accuracy:.2f}, Completion Time: {completion_time:.2f}, Deaths: {deaths}")
        print(f"  Current Difficulty: {difficulty:.2f}")

        difficulty = adjust_difficulty(player_skill, accuracy, completion_time, deaths, difficulty, model)
        print(f"  Adjusted Difficulty: {difficulty:.2f}\n")

        # Update player skill based on performance (simple example)
        player_skill += (accuracy - 0.5) * 0.1

        # Keep skill within 0-1 range
        player_skill = max(0, min(1, player_skill))

gameplay_loop(model)

Round 1:
  Accuracy: 1.02, Completion Time: 11.09, Deaths: 0
  Current Difficulty: 0.50
  Adjusted Difficulty: 0.43

Round 2:
  Accuracy: 0.90, Completion Time: 11.56, Deaths: 0
  Current Difficulty: 0.43
  Adjusted Difficulty: 0.40

Round 3:
  Accuracy: 0.61, Completion Time: 13.61, Deaths: 0
  Current Difficulty: 0.40
  Adjusted Difficulty: 0.38

Round 4:
  Accuracy: 0.61, Completion Time: 10.57, Deaths: 0
  Current Difficulty: 0.38
  Adjusted Difficulty: 0.53

Round 5:
  Accuracy: 1.10, Completion Time: 6.93, Deaths: 0
  Current Difficulty: 0.53
  Adjusted Difficulty: 0.61

Round 6:
  Accuracy: 0.99, Completion Time: 6.55, Deaths: 1
  Current Difficulty: 0.61
  Adjusted Difficulty: 0.69

Round 7:
  Accuracy: 0.75, Completion Time: 6.20, Deaths: 0
  Current Difficulty: 0.69
  Adjusted Difficulty: 0.75

Round 8:
  Accuracy: 0.74, Completion Time: 5.95, Deaths: 0
  Current Difficulty: 0.75
  Adjusted Difficulty: 0.80

Round 9:
  Accuracy: 0.86, Completion Time: 7.34, Deaths: 0
  Curren